# Context managers

## As explained in MCS 275

In Spring 2024, instructor David Dumas

### This is a sub-subheading

> This text cell is longer and more complicated than necessary so that we can demonstrate a bit of [Markdown](https://www.markdownguide.org/basic-syntax/).

Reasons why you would use context managers:
1. To make explicit what part of a program holds a resource
1. To ensure cleanup happens even if an exception occurs during use of a resource
1. As an alternative to object destructors that include stronger guarantees about when they will be called.

Special methods this involves:
* `__enter__`
* `__exit__`

Python syntax involved:
```python
with ClassName(args) as varname:
    stuff
```

Based on [Lecture 7 of MCS 275, Spring 2024](https://www.dumas.io/teaching/2024/spring/mcs275/slides/lecture7.html).  (That's how you make links in Markdown, by the way.)

### By the way, math!

Were you aware of this integral fact?
$$\int_0^\infty e^{-x^2} \, dx = \frac{\sqrt{\pi}}{2}$$
Isn't it surprising that $\pi$ would show up in the answer?

This is to show that Markdown supports the LaTeX language for mathematical expressions.  You can find a quick intro to mathematical expressions in LaTeX in the [documentation for Overleaf, an online service for preparing LaTeX documents](https://www.overleaf.com/learn/latex/Mathematical_expressions).


## Manual cleanup way to do file I/O

In [2]:
fp = open("example.txt","w") # acquire
fp.write("Hello world!\n")   # use
fp.close()                   # release

## Automatic cleanup using `with`

In [ ]:
with open("example.txt","w") as fp:  # acquire
    fp.write("Hello world!\n")       # use

# implicit release as soon as indented block ends.

## Custom context manager

A class whose objects can be used in `with`-blocks is called a **Context Manager**.

To be a context manager, a class just needs two methods:
* `__enter__(self):` - Should perform acquisition, called when entering `with` block
* `__exit__(self,exc_type,exc,tb):` - Should perform cleanup, called when exiting `with` block whether normally or because of an exception.  The three argments after `self` will be `None` if the `with`-block ended normally, and otherwise will contain information about any exception that happened.

A context manager will *usually* also have a constructor, since that will be called when the object is created (often on the same line as the start of the `with`-block).

Let's make a context manager that temporarily changes the current working directory.  Upon exit from the with-block, it will restore the previous current working directory.

In [4]:
import os

In [5]:
class TemporaryChangeWorkingDirectory:
    "Context manager that saves CWD, changes to a given one, then restores original on exit"
    def __init__(self,dirname):
        "Initialize a context manager that, on entry to with-block, changes to `dirname`"
        self.dirname = dirname

    def __enter__(self):
        "Enter a with-block: Save the current working dir and change to `self.dirname`"
        print("Entering the with-block, saving and changing CWD")
        self.oldcwd = os.getcwd() # save
        os.chdir(self.dirname)    # change
        
    def __exit__(self,exc_type,exc,tb):
        "Restore previous current working dir"
        print("Exiting the with-block, restoring previous CWD")
        os.chdir(self.oldcwd)

In [7]:
print("Before: ",os.getcwd())

with TemporaryChangeWorkingDirectory("/tmp"):
    print("During: ",os.getcwd())
    print("Other work done inside the with-block")
    
print("After: ",os.getcwd())

Before:  /home/ddumas
Entering the with-block, saving and changing CWD
During:  /tmp
Other work done inside the with-block
Exiting the with-block, restoring previous CWD
After:  /home/ddumas


## Cleanup still happens even if there is an exception

In [8]:
print("Before: ",os.getcwd())

with TemporaryChangeWorkingDirectory("/tmp"):
    print("During: ",os.getcwd())
    1/0    
print("After: ",os.getcwd())

Before:  /home/ddumas
Entering the with-block, saving and changing CWD
During:  /tmp
Exiting the with-block, restoring previous CWD


ZeroDivisionError: division by zero

## Returning a value from `__enter__`

If `__enter__` returns a value, it gets assigned to the variable name appearing after `as`.

So:
```python
with ClassName() as x:
```
is an auto-cleanup version of
```python
x = ClassName()
```
while
```python
with ClassName():
```
is an auto-cleanup version of
```
ClassName()
```

For example, `open()` is a context manager which returns a file object.